In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

from utils import (
    generatePEDModelsSku,
    generatePEDModelsByPriceSku
)

import warnings
warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('SALES-full.csv', sep=';')

In [3]:
df1['curr_date'] = pd.to_datetime(df1['curr_date'])

# SKU

In [4]:
index = ['SKU_id']

In [5]:
df = df1[0:40000000]
df.shape

(40000000, 9)

In [6]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [7]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [8]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [9]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [10]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [11]:
popular_pairs = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,counts
0,562,162700
1,1358,162598
2,1068,160718
3,4452,160141
4,14094,160031
...,...,...
420,12888,1344
421,8580,1291
422,14424,1050
423,4660,1021


In [12]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,counts
0,1358,23917
1,14094,23875
2,7341,23855
3,1068,23826
4,562,23819
...,...,...
420,12888,207
421,8580,190
422,4259,160
423,14424,159


In [13]:
sample_data_day_0to40 = generatePEDModelsSku(df, amount=num_day)
sample_data_day_0to40['Гранулярность эластичности'] = 'SKU'
sample_data_day_0to40['Гранулярность trainset'] = 'SKU|Day'

In [14]:
sample_data_day_0to40_p = generatePEDModelsByPriceSku(df, amount=num_day)
sample_data_day_0to40_p['Гранулярность эластичности'] = 'SKU'
sample_data_day_0to40_p['Гранулярность trainset'] = 'SKU|Day'

In [15]:
sample_data_week_0to40 = generatePEDModelsSku(df_w, date_type='week', amount=num_week)
sample_data_week_0to40['Гранулярность эластичности'] = 'SKU'
sample_data_week_0to40['Гранулярность trainset'] = 'SKU|Week'

In [16]:
sample_data_week_0to40_p = generatePEDModelsByPriceSku(df_w, date_type='week', amount=num_week)
sample_data_week_0to40_p['Гранулярность эластичности'] = 'SKU'
sample_data_week_0to40_p['Гранулярность trainset'] = 'SKU|Week'

In [17]:
sample_data_day_0to40

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,562,0.452608,0.015261,0.515447,log-log,0.452608,1.0,22,30,SKU,SKU|Day
1,562,0.179902,0.004667,0.675319,log-log,0.179902,1.0,28,40,SKU,SKU|Day
2,562,-1.765607,0.106413,0.020780,log-log,-1.765607,5.0,37,50,SKU,SKU|Day
3,562,-1.099432,0.069014,0.042574,log-log,-1.099432,5.0,41,60,SKU,SKU|Day
4,562,-1.241509,0.085799,0.013863,log-log,-1.241509,6.0,46,70,SKU,SKU|Day
...,...,...,...,...,...,...,...,...,...,...,...
22949,9708,0.042886,0.001755,0.349888,log-log,0.042886,27.0,6,500,SKU,SKU|Day
22950,9708,0.045862,0.001944,0.280887,log-log,0.045862,34.0,6,600,SKU,SKU|Day
22951,9708,0.060551,0.003238,0.132544,log-log,0.060551,42.0,6,700,SKU,SKU|Day
22952,9708,0.051611,0.002576,0.151478,log-log,0.051611,44.0,6,800,SKU,SKU|Day


In [18]:
sample_data_day_0to40_p

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,562,36.727089,0.000033,6.203374e-01,log-log,36.727089,150.0,1,SKU,SKU|Day
1,562,0.164178,0.005414,4.566325e-19,log-log,0.164178,173.0,2,SKU,SKU|Day
2,562,0.157662,0.007299,3.564233e-34,log-log,0.157662,179.0,3,SKU,SKU|Day
3,562,0.606305,0.009691,6.373304e-55,log-log,0.606305,2147.0,4,SKU,SKU|Day
4,562,0.738231,0.016015,1.787942e-105,log-log,0.738231,2151.0,5,SKU,SKU|Day
...,...,...,...,...,...,...,...,...,...,...
52337,9708,0.083098,0.005112,3.850938e-02,log-log,0.083098,50.0,2,SKU,SKU|Day
52338,9708,0.062325,0.003706,6.948328e-02,log-log,0.062325,53.0,3,SKU,SKU|Day
52339,9708,0.061725,0.003483,7.217499e-02,log-log,0.061725,55.0,4,SKU,SKU|Day
52340,9708,0.058464,0.003113,8.338462e-02,log-log,0.058464,56.0,5,SKU,SKU|Day


In [19]:
sample_data_week_0to40

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,1358,-0.403381,0.067880,0.467210,log-log,-0.403381,0.857143,10,10,SKU,SKU|Week
1,1358,-0.262149,0.030686,0.460092,log-log,-0.262149,1.714286,17,20,SKU,SKU|Week
2,1358,-0.324295,0.031162,0.350741,log-log,-0.324295,4.142857,26,30,SKU,SKU|Week
3,1358,-0.318219,0.031545,0.272894,log-log,-0.318219,4.738095,31,40,SKU,SKU|Week
4,1358,-0.248400,0.022103,0.302821,log-log,-0.248400,5.595238,37,50,SKU,SKU|Week
...,...,...,...,...,...,...,...,...,...,...,...
14082,9708,0.085325,0.009841,0.450789,log-log,0.085325,4.571429,8,60,SKU,SKU|Week
14083,9708,0.094002,0.012627,0.354368,log-log,0.094002,5.000000,8,70,SKU,SKU|Week
14084,9708,0.051990,0.004221,0.566925,log-log,0.051990,5.285714,8,80,SKU,SKU|Week
14085,9708,0.068063,0.008035,0.400817,log-log,0.068063,6.023810,9,90,SKU,SKU|Week


In [20]:
sample_data_week_0to40_p

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,1358,116.387367,0.000891,8.497090e-02,log-log,116.387367,541.078571,1,SKU,SKU|Week
1,1358,-1.639749,0.047739,1.210449e-46,log-log,-1.639749,580.678571,2,SKU,SKU|Week
2,1358,-1.261165,0.039109,7.929096e-46,log-log,-1.261165,735.340476,3,SKU,SKU|Week
3,1358,-0.618570,0.013644,9.040875e-19,log-log,-0.618570,799.238095,4,SKU,SKU|Week
4,1358,-0.606659,0.013082,7.108607e-20,log-log,-0.606659,874.828571,5,SKU,SKU|Week
...,...,...,...,...,...,...,...,...,...,...
58962,9708,0.053184,0.008431,2.823450e-01,log-log,0.053184,6.714286,7,SKU,SKU|Week
58963,9708,0.074032,0.012971,1.787137e-01,log-log,0.074032,7.571429,8,SKU,SKU|Week
58964,9708,0.086001,0.016771,1.231878e-01,log-log,0.086001,8.000000,9,SKU,SKU|Week
58965,9708,0.081644,0.015326,1.393220e-01,log-log,0.081644,8.000000,10,SKU,SKU|Week


In [21]:
df = df1[40000000:80000000]
df.shape

(40000000, 9)

In [22]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [23]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [24]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [25]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [26]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [27]:
popular_pairs = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,counts
0,61591,161240
1,20695,161187
2,27900,161127
3,65319,160863
4,73233,160178
...,...,...
422,32847,1362
423,66776,1357
424,19249,1204
425,32172,1158


In [28]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,counts
0,25164,23842
1,27900,23806
2,73233,23795
3,17584,23744
4,19703,23736
...,...,...
423,32847,205
424,19249,181
425,32172,180
426,66779,160


In [29]:
sample_data_day_40to80 = generatePEDModelsSku(df, amount=num_day)
sample_data_day_40to80['Гранулярность эластичности'] = 'SKU'
sample_data_day_40to80['Гранулярность trainset'] = 'SKU|Day'

In [30]:
sample_data_day_40to80_p = generatePEDModelsByPriceSku(df, amount=num_day)
sample_data_day_40to80_p['Гранулярность эластичности'] = 'SKU'
sample_data_day_40to80_p['Гранулярность trainset'] = 'SKU|Day'

In [31]:
sample_data_week_40to80 = generatePEDModelsSku(df_w, date_type='week', amount=num_week)
sample_data_week_40to80['Гранулярность эластичности'] = 'SKU'
sample_data_week_40to80['Гранулярность trainset'] = 'SKU|Week'

In [32]:
sample_data_week_40to80_p = generatePEDModelsByPriceSku(df_w, date_type='week', amount=num_week)
sample_data_week_40to80_p['Гранулярность эластичности'] = 'SKU'
sample_data_week_40to80_p['Гранулярность trainset'] = 'SKU|Week'

In [33]:
df = df1[80000000:120000000]
df.shape

(40000000, 9)

In [34]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [35]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [36]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [37]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [38]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [39]:
popular_pairs = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,counts
0,97689,162916
1,97641,161690
2,96261,160339
3,98683,160153
4,99251,159790
...,...,...
459,106469,1354
460,112072,1302
461,103871,1212
462,95866,1195


In [40]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,counts
0,97689,23959
1,96261,23834
2,99251,23823
3,98683,23785
4,97641,23708
...,...,...
460,106469,200
461,99672,189
462,95866,179
463,103871,179


In [41]:
sample_data_day_80to120 = generatePEDModelsSku(df, amount=num_day)
sample_data_day_80to120['Гранулярность эластичности'] = 'SKU'
sample_data_day_80to120['Гранулярность trainset'] = 'SKU|Day'

In [42]:
sample_data_day_80to120_p = generatePEDModelsByPriceSku(df, amount=num_day)
sample_data_day_80to120_p['Гранулярность эластичности'] = 'SKU'
sample_data_day_80to120_p['Гранулярность trainset'] = 'SKU|Day'

In [43]:
sample_data_week_80to120 = generatePEDModelsSku(df_w, date_type='week', amount=num_week)
sample_data_week_80to120['Гранулярность эластичности'] = 'SKU'
sample_data_week_80to120['Гранулярность trainset'] = 'SKU|Week'

In [44]:
sample_data_week_80to120_p = generatePEDModelsByPriceSku(df_w, date_type='week', amount=num_week)
sample_data_week_80to120_p['Гранулярность эластичности'] = 'SKU'
sample_data_week_80to120_p['Гранулярность trainset'] = 'SKU|Week'

In [45]:
df = df1[120000000:160000000]
df.shape

(40000000, 9)

In [46]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [47]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [48]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [49]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [50]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [51]:
popular_pairs = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,counts
0,132714,158231
1,132643,157432
2,132154,156881
3,141821,156496
4,139209,156121
...,...,...
510,137701,1510
511,126330,1435
512,132307,1339
513,143571,1077


In [52]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,counts
0,132714,23600
1,141714,23579
2,139209,23487
3,132154,23430
4,141821,23397
...,...,...
512,132307,200
513,141743,191
514,143571,161
515,145007,158


In [53]:
sample_data_day_120to160 = generatePEDModelsSku(df, amount=num_day)
sample_data_day_120to160['Гранулярность эластичности'] = 'SKU'
sample_data_day_120to160['Гранулярность trainset'] = 'SKU|Day'

In [54]:
sample_data_day_120to160_p = generatePEDModelsByPriceSku(df, amount=num_day)
sample_data_day_120to160_p['Гранулярность эластичности'] = 'SKU'
sample_data_day_120to160_p['Гранулярность trainset'] = 'SKU|Day'

In [55]:
sample_data_week_120to160 = generatePEDModelsSku(df_w, date_type='week', amount=num_week)
sample_data_week_120to160['Гранулярность эластичности'] = 'SKU'
sample_data_week_120to160['Гранулярность trainset'] = 'SKU|Week'

In [56]:
sample_data_week_120to160_p = generatePEDModelsByPriceSku(df_w, date_type='week', amount=num_week)
sample_data_week_120to160_p['Гранулярность эластичности'] = 'SKU'
sample_data_week_120to160_p['Гранулярность trainset'] = 'SKU|Week'

In [57]:
df = df1[160000000:183051540]
df.shape

(23051540, 9)

In [58]:
df.rename(columns={'Code_AP' : 'SKU_id',
                   'store_id' : 'Store_id',
                   'curr_date' : 'Date',
                   'stock' : 'Stock',
                   'sales_regular' : 'Sales',
                   'price_regular' : 'Regular_Price'
                    }, inplace=True)
df.drop(['sales_promo', 'price_promo', 'flag_promo'], axis=1, inplace=True)

In [59]:
df = df.loc[df['Stock'] > df['Sales']]
df['Demand'] = df.loc[df['Stock'] > 0, 'Sales']
df.set_index(['SKU_id', 'Store_id', 'Date'], inplace=True)

In [60]:
level_values = df.index.get_level_values
df_w = (df.groupby([level_values(i) for i in [0,1]]
                      +[pd.Grouper(freq='W', level=-1)]).mean())
# агрегация исходных данные в разрезе SKU|Store|Week

In [61]:
df_w.reset_index(inplace=True)
df.reset_index(inplace=True)

In [62]:
df['Round_Price'] = round(df['Regular_Price'])
df_w['Round_Price'] = round(df_w['Regular_Price'])

In [63]:
popular_pairs = df.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_day = popular_pairs[popular_pairs['counts'] >= 1000].shape[0]
popular_pairs[popular_pairs['counts'] >= 1000]

,SKU_id,counts
0,363196,155286
1,363195,155043
2,145861,148802
3,146841,147857
4,145860,145491
...,...,...
466,221159,1188
467,206843,1102
468,213028,1061
469,320072,1019


In [64]:
popular_pairs = df_w.groupby(index).size().reset_index(name='counts').sort_values(by='counts', ascending=False).reset_index(drop=True)
num_week = popular_pairs[popular_pairs['counts'] >= 150].shape[0]
popular_pairs[popular_pairs['counts'] >= 150]

,SKU_id,counts
0,363196,23165
1,363195,22970
2,145861,22449
3,146841,21984
4,145860,21963
...,...,...
465,151267,201
466,221159,183
467,321217,182
468,206843,165


In [65]:
sample_data_day_160to_end = generatePEDModelsSku(df, amount=num_day)
sample_data_day_160to_end['Гранулярность эластичности'] = 'SKU'
sample_data_day_160to_end['Гранулярность trainset'] = 'SKU|Day'

In [66]:
sample_data_day_160to_end_p = generatePEDModelsByPriceSku(df, amount=num_day)
sample_data_day_160to_end_p['Гранулярность эластичности'] = 'SKU'
sample_data_day_160to_end_p['Гранулярность trainset'] = 'SKU|Day'

In [67]:
sample_data_week_160to_end = generatePEDModelsSku(df_w, date_type='week', amount=num_week)
sample_data_week_160to_end['Гранулярность эластичности'] = 'SKU'
sample_data_week_160to_end['Гранулярность trainset'] = 'SKU|Week'

In [68]:
sample_data_week_160to_end_p = generatePEDModelsByPriceSku(df_w, date_type='week', amount=num_week)
sample_data_week_160to_end_p['Гранулярность эластичности'] = 'SKU'
sample_data_week_160to_end_p['Гранулярность trainset'] = 'SKU|Week'

In [69]:
sample_data_day = sample_data_day_0to40.append([sample_data_day_40to80, sample_data_day_80to120, sample_data_day_120to160, sample_data_day_160to_end])
sample_data_day.reset_index(inplace=True, drop=True)
sample_data_day.to_csv('models_sku/sample_data_day.csv', index=False)
sample_data_day

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,562,0.452608,0.015261,0.515447,log-log,0.452608,1.0,22,30,SKU,SKU|Day
1,562,0.179902,0.004667,0.675319,log-log,0.179902,1.0,28,40,SKU,SKU|Day
2,562,-1.765607,0.106413,0.020780,log-log,-1.765607,5.0,37,50,SKU,SKU|Day
3,562,-1.099432,0.069014,0.042574,log-log,-1.099432,5.0,41,60,SKU,SKU|Day
4,562,-1.241509,0.085799,0.013863,log-log,-1.241509,6.0,46,70,SKU,SKU|Day
...,...,...,...,...,...,...,...,...,...,...,...
113029,319939,-0.003417,0.000045,0.881676,log-log,-0.003417,1.0,25,500,SKU,SKU|Day
113030,319939,-0.002888,0.000040,0.877690,log-log,-0.002888,1.0,25,600,SKU,SKU|Day
113031,319939,-0.010972,0.000353,0.619958,log-log,-0.010972,2.0,25,700,SKU,SKU|Day
113032,319939,-0.009308,0.000311,0.618210,log-log,-0.009308,2.0,25,800,SKU,SKU|Day


In [70]:
sample_data_day_p = sample_data_day_0to40_p.append([sample_data_day_40to80_p, sample_data_day_80to120_p, sample_data_day_120to160_p, sample_data_day_160to_end_p])
sample_data_day_p.reset_index(inplace=True, drop=True)
sample_data_day_p.to_csv('models_sku/sample_data_day_p.csv', index=False)
sample_data_day_p

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,562,36.727089,0.000033,6.203374e-01,log-log,36.727089,150.0,1,SKU,SKU|Day
1,562,0.164178,0.005414,4.566325e-19,log-log,0.164178,173.0,2,SKU,SKU|Day
2,562,0.157662,0.007299,3.564233e-34,log-log,0.157662,179.0,3,SKU,SKU|Day
3,562,0.606305,0.009691,6.373304e-55,log-log,0.606305,2147.0,4,SKU,SKU|Day
4,562,0.738231,0.016015,1.787942e-105,log-log,0.738231,2151.0,5,SKU,SKU|Day
...,...,...,...,...,...,...,...,...,...,...
290930,319939,-0.007911,0.000270,6.146750e-01,log-log,-0.007911,2.0,21,SKU,SKU|Day
290931,319939,-0.007921,0.000271,6.138362e-01,log-log,-0.007921,2.0,22,SKU,SKU|Day
290932,319939,-0.007931,0.000272,6.131582e-01,log-log,-0.007931,2.0,23,SKU,SKU|Day
290933,319939,-0.007938,0.000272,6.126298e-01,log-log,-0.007938,2.0,24,SKU,SKU|Day


In [71]:
sample_data_week = sample_data_week_0to40.append([sample_data_week_40to80, sample_data_week_80to120, sample_data_week_120to160, sample_data_week_160to_end])
sample_data_week.reset_index(inplace=True, drop=True)
sample_data_week.to_csv('models_sku/sample_data_week.csv', index=False)
sample_data_week

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,Num_diff_prices,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,1358,-0.403381,0.067880,0.467210,log-log,-0.403381,0.857143,10,10,SKU,SKU|Week
1,1358,-0.262149,0.030686,0.460092,log-log,-0.262149,1.714286,17,20,SKU,SKU|Week
2,1358,-0.324295,0.031162,0.350741,log-log,-0.324295,4.142857,26,30,SKU,SKU|Week
3,1358,-0.318219,0.031545,0.272894,log-log,-0.318219,4.738095,31,40,SKU,SKU|Week
4,1358,-0.248400,0.022103,0.302821,log-log,-0.248400,5.595238,37,50,SKU,SKU|Week
...,...,...,...,...,...,...,...,...,...,...,...
69649,151090,-0.185130,0.004100,0.626952,log-log,-0.185130,1.142857,37,60,SKU,SKU|Week
69650,151090,-0.155117,0.004099,0.598495,log-log,-0.155117,1.142857,40,70,SKU,SKU|Week
69651,151090,-0.118945,0.003639,0.595027,log-log,-0.118945,1.142857,43,80,SKU,SKU|Week
69652,151090,-0.093709,0.002356,0.649576,log-log,-0.093709,1.142857,46,90,SKU,SKU|Week


In [72]:
sample_data_week_p = sample_data_week_0to40_p.append([sample_data_week_40to80_p, sample_data_week_80to120_p, sample_data_week_120to160_p, sample_data_week_160to_end_p])
sample_data_week_p.reset_index(inplace=True, drop=True)
sample_data_week_p.to_csv('models_sku/sample_data_week_p.csv', index=False)
sample_data_week_p

,SKU,Coefficient,R-squared,T-test-p-value,Model-type,Elasticity,Qty,SampleSize,Гранулярность эластичности,Гранулярность trainset
0,1358,116.387367,0.000891,8.497090e-02,log-log,116.387367,541.078571,1,SKU,SKU|Week
1,1358,-1.639749,0.047739,1.210449e-46,log-log,-1.639749,580.678571,2,SKU,SKU|Week
2,1358,-1.261165,0.039109,7.929096e-46,log-log,-1.261165,735.340476,3,SKU,SKU|Week
3,1358,-0.618570,0.013644,9.040875e-19,log-log,-0.618570,799.238095,4,SKU,SKU|Week
4,1358,-0.606659,0.013082,7.108607e-20,log-log,-0.606659,874.828571,5,SKU,SKU|Week
...,...,...,...,...,...,...,...,...,...,...
334804,151090,-0.174634,0.010618,2.190728e-01,log-log,-0.174634,2.166667,65,SKU,SKU|Week
334805,151090,-0.171009,0.010234,2.260014e-01,log-log,-0.171009,2.166667,66,SKU,SKU|Week
334806,151090,-0.170498,0.010170,2.258345e-01,log-log,-0.170498,2.166667,67,SKU,SKU|Week
334807,151090,-0.170277,0.010140,2.249202e-01,log-log,-0.170277,2.166667,68,SKU,SKU|Week
